In [22]:
start_date = '2012-01-01'
parser = lambda x: pd.datetime.strptime(x, "%Y.%m.%d %H:%M:%S")
df = pd.read_csv('EURUSD_RECENTE.csv', sep=';', header=0,parse_dates=['time'],date_parser=parser)
avanco_da_rede = 6099
indice_total_treino = 0


features = 4
timestamp = 2
epochs = 50
janela = 7
x_train = []
y_train = []
x_train = None
y_train = None

mask = (df['time'] > start_date) & (df['time'] <= periodo_aprendizado )
df_train = df.loc[mask]

df_train = df_train.iloc[:,1:5]

# print(df_train)

indice_total_treino = len(df_train)

df_test      = df.iloc[ indice_total_treino:indice_total_treino+avanco_da_rede,1:5]
df_test_orig = df.iloc[ indice_total_treino:indice_total_treino+avanco_da_rede,0:5]

# print(df_test_orig)
#periodo_aprendizado = df.iloc[indice_total_treino+avanco_da_rede-3,0]

#Escala nas features
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
df_train = sc.fit_transform(df_train)
#Formação do x_train
x_train = []
y_train = []
length = len(df_train)
for i in range(timestamp, length):
  x_train.append( df_train[i-timestamp:i,:])
  y_train.append( df_train[i,features-1])

x_train = np.array(x_train)
y_train = np.array(y_train)

print(x_train[0])

x_train = np.reshape(x_train, (x_train.shape[0], timestamp, features))
  
# # ###Treinamento da rede neural
model = None
model = Sequential()
# model.add(LSTM(units = 480, return_sequences = True, input_shape=( timestamp , features) ) ) 
model.add(LSTM(units = 480, return_sequences = False, input_shape=( timestamp , features) )) 
#model.add(Dropout(0.1))
#model.add(Dense(units = 1)) 
#model.compile(optimizer = 'adam', loss = 'mean_squared_error')
#model.add(LSTM(50, input_shape=(44725, x_train.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
model.fit(x_train, y_train, epochs = epochs)
model.save('LSTM')
#import tensorflow as tf
#model = tf.keras.models.load_model('drive/My Drive/TCC/modelo/LSTM')
#df_test_close = df_test_close.reshape(-1, 1)
#df_test = sc.fit_transform(df_test)
from sklearn.preprocessing import MinMaxScaler
sc1 = MinMaxScaler()
sc1.fit(df_train_orig.iloc[:,features-1].values.reshape(-1,1))
df_test = sc.transform(df_test)
x_test =    []
y_test =    []
y_max  =    []
y_min  =    [] 
open_test = []
data_test = []

####Teste da rede neural##################

#pegando maximos e minimos para testes com a previsao


length = len(df_test)
for i in range(timestamp, length):
  x_test.append( df_test[i-timestamp:i,:])
  y_test.append( df_test[i,features-1])
  y_max.append( df_test_orig.iloc[i,3])
  y_min.append( df_test_orig.iloc[i,2])
  open_test.append( df_test_orig.iloc[i-1,4] )
  data_test.append( df_test_orig.iloc[i,0])


x_test = np.array(x_test)
y_test = np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], timestamp, features))

#Criando variavel Y_pred para receber as previsões
y_pred = []
y_pred = model.predict(x_test)

#Invertendo a escala para valores reais
predicted_price = sc1.inverse_transform(y_pred)
y_test = sc1.inverse_transform(y_test.reshape(-1,1))

# shift = y_test[0] - predicted_price[0]
# predicted_price = predicted_price + shift

janela = len(predicted_price)

previsao = pd.DataFrame(data=predicted_price , columns=['Previsão'])
numero_real = pd.DataFrame(data=y_test , columns=['Fecahmaneto'])
preco_de_abertura = pd.DataFrame(data=open_test , columns=['Abertura'])
maxima = pd.DataFrame(data=y_max , columns=['Máxima'])
minima = pd.DataFrame(data=y_min , columns=['Mínima'])
# frames = [ preco_de_abertura, maxima, minima, numero_real, previsao,resumo_acumulado]
# result = pd.concat(frames, axis=1, join='inner')
data_test_pd =  pd.DataFrame(data=data_test , columns=['Data'])
frames_to_csv= [ data_test_pd,maxima, minima, previsao]
result_to_csv = pd.concat( frames_to_csv, axis=1, join='inner')

result_to_csv.to_csv('previsoes_anuais.csv', mode='a', header=False,index=False)




         open1    high1     low1   close1
55868  1.22139  1.22110  1.22258  1.22253
55869  1.22239  1.22158  1.22267  1.22177
55870  1.22135  1.22119  1.22245  1.22135
55871  1.22133  1.21814  1.22165  1.21863
55872  1.21863  1.21805  1.21953  1.21880
...        ...      ...      ...      ...
56224  1.21486  1.21289  1.21528  1.21345
56225  1.21345  1.21287  1.21432  1.21317
56226  1.21317  1.21268  1.21358  1.21286
56227  1.21290  1.21278  1.21341  1.21300
56228  1.21300  1.21297  1.21386  1.21340

[361 rows x 4 columns]
[[0.74607047 0.74970203 0.73971128 0.74625624]
 [0.74607047 0.74388114 0.73915924 0.74043261]]
Epoch 1/2
1746/1746 [==============================] - 16s 9ms/step - loss: 0.0214
Epoch 2/2
1746/1746 [==============================] - 16s 9ms/step - loss: 0.0046
INFO:tensorflow:Assets written to: drive/My Drive/TCC/modelo/LSTM/assets


In [18]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  multiple                  931200    
_________________________________________________________________
dense (Dense)                multiple                  481       
Total params: 931,681
Trainable params: 931,681
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
indice_total_treino = len(df_train)

df_test      = df.iloc[ indice_total_treino:indice_total_treino+avanco_da_rede,1:5]
df_test_orig = df.iloc[ indice_total_treino:indice_total_treino+avanco_da_rede,0:5]

from sklearn.preprocessing import MinMaxScaler
sc1 = MinMaxScaler()
sc1.fit(df_train_orig.iloc[:,features-1].values.reshape(-1,1))
df_test = sc.transform(df_test)
x_test =    []
y_test =    []
y_max  =    []
y_min  =    [] 
open_test = []
data_test = []

####Teste da rede neural##################

#pegando maximos e minimos para testes com a previsao


length = len(df_test)
for i in range(timestamp, length):
  x_test.append( df_test[i-timestamp:i,:])
  y_test.append( df_test[i,features-1])
  y_max.append( df_test_orig.iloc[i,3])
  y_min.append( df_test_orig.iloc[i,2])
  open_test.append( df_test_orig.iloc[i-1,4] )
  data_test.append( df_test_orig.iloc[i,0])


x_test = np.array(x_test)
y_test = np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], timestamp, features))

#Criando variavel Y_pred para receber as previsões
y_pred = []
y_pred = model.predict(x_test)

#Invertendo a escala para valores reais
predicted_price = sc1.inverse_transform(y_pred)
y_test = sc1.inverse_transform(y_test.reshape(-1,1))

# shift = y_test[0] - predicted_price[0]
# predicted_price = predicted_price + shift

janela = len(predicted_price)

previsao = pd.DataFrame(data=predicted_price , columns=['Previsão'])
numero_real = pd.DataFrame(data=y_test , columns=['Fecahmaneto'])
#y_max_pd = pd.DataFrame(data=y_max, columns=['Máxima'])
preco_de_abertura = pd.DataFrame(data=open_test , columns=['Abertura'])
maxima = pd.DataFrame(data=y_max , columns=['Máxima'])
minima = pd.DataFrame(data=y_min , columns=['Mínima'])
frames = [ preco_de_abertura, maxima, minima, numero_real, previsao]
result = pd.concat(frames, axis=1, join='inner')

#Graficos de previsao
plt.plot(y_test[:janela], color='red', label='Preço Atual das Ações')
plt.plot(result.index.values[:janela], result.iloc[:janela,4].values , '.')
plt.fill_between(result.index.values[:janela], result.iloc[:janela,1].values, result.iloc[:janela,2], alpha=0.5)
plt.title('Previsão de Preço de Ações')
plt.xlabel('Tempo')
plt.ylabel('Preço das Ações')
plt.legend()
plt.show()


#####Calculo de acertos da previsao####################
acumulador = 0
lucro_prejuizo = []
resumo_acumulado = []
acertos = 0
erros   = 0
resumo  = []
portcentagem = 0

for i in range (0, janela):
    soma = 0
    if predicted_price[i] in pd.Interval(left=y_min[i], right=y_max[i]):
      soma = abs( np.round(predicted_price[i],5) - np.around(open_test[i], 5 ))
      soma = int(soma*100000)
      acertos = acertos + 1
    else:
      if y_test[i] > open_test[i]:
        if predicted_price[i] > open_test[i]:
          soma = abs( np.round(y_test[i], 5) - np.round(open_test[i], 5) )
          soma = int(soma*100000)
          acertos = acertos + 1
        else:
          soma = np.round(open_test[i], 5) - np.round(y_test[i], 5)
          soma = int(soma*100000)
          erros = erros + 1
      else:
        if predicted_price[i] < open_test[i]:
          soma = abs( np.round(y_test[i], 5) - np.round(open_test[i], 5))
          soma = int(soma*100000)
          acertos = acertos + 1
        else:
          soma = np.round(y_test[i], 5) - np.round(open_test[i], 5)
          soma = int(soma*100000)
          erros = erros + 1
    acumulador = acumulador + soma
    resumo_acumulado.append(acumulador)

portcentagem = acertos*100/janela
print("Acertos: ", acertos)
print("Erros: ", erros)
print("Pontos Ganhos: ", acumulador)
print("%", portcentagem)
print("Meses em teste estudados: ", int(len(predicted_price)/janela))
# resumo = pd.DataFrame(data=lucro_prejuizo , columns=['Resumo'])
resumo_acumulado = pd.DataFrame(data=resumo_acumulado , columns=['Resumo Acumulado '])
#return acumulador



plt.plot(resumo_acumulado, color='blue', label='Preço Atual das Ações')
plt.title('Resumo Acumulado')
plt.xlabel('Tempo')
plt.ylabel('Preço das Ações')
plt.legend()
plt.show()
plt.title('Preço final')
plt.xlabel('Tempo')
plt.ylabel('Preço das Ações')
plt.plot(y_test, color='red', label='Preço Atual das Ações')
plt.show()
# frames = [ preco_de_abertura, maxima, minima, numero_real, previsao,resumo_acumulado]
# result = pd.concat(frames, axis=1, join='inner')



In [ ]:

#Calculos  para previsao
previsao = pd.DataFrame(data=predicted_price , columns=['Previsão'])
numero_real = pd.DataFrame(data=y_test , columns=['Fecahmaneto'])
#y_max_pd = pd.DataFrame(data=y_max, columns=['Máxima'])
preco_de_abertura = pd.DataFrame(data=open_test , columns=['Abertura'])
maxima = pd.DataFrame(data=y_max , columns=['Máxima'])
minima = pd.DataFrame(data=y_min , columns=['Mínima'])
frames = [ preco_de_abertura, maxima, minima, numero_real, previsao]
result = pd.concat(frames, axis=1, join='inner')

#Graficos de previsao
plt.plot(y_test[:janela], color='red', label='Preço Atual das Ações')
plt.plot(result.index.values[:janela], result.iloc[:janela,4].values , '.')
plt.fill_between(result.index.values[:janela], result.iloc[:janela,1].values, result.iloc[:janela,2], alpha=0.5)
plt.title('Previsão de Preço de Ações')
plt.xlabel('Tempo')
plt.ylabel('Preço das Ações')
plt.legend()
plt.show()


#####Calculo de acertos da previsao####################
acumulador = 0
lucro_prejuizo = []
resumo_acumulado = []
acertos = 0
erros   = 0
resumo  = []
portcentagem = 0

for i in range (0, janela):
    soma = 0
    if predicted_price[i] in pd.Interval(left=y_min[i], right=y_max[i]):
      soma = abs( np.round(predicted_price[i],5) - np.around(open_test[i], 5 ))
      soma = int(soma*100000)
      acertos = acertos + 1
    else:
      if y_test[i] > open_test[i]:
        if predicted_price[i] > open_test[i]:
          soma = abs( np.round(y_test[i], 5) - np.round(open_test[i], 5) )
          soma = int(soma*100000)
          acertos = acertos + 1
        else:
          soma = np.round(open_test[i], 5) - np.round(y_test[i], 5)
          soma = int(soma*100000)
          erros = erros + 1
      else:
        if predicted_price[i] < open_test[i]:
          soma = abs( np.round(y_test[i], 5) - np.round(open_test[i], 5))
          soma = int(soma*100000)
          acertos = acertos + 1
        else:
          soma = np.round(y_test[i], 5) - np.round(open_test[i], 5)
          soma = int(soma*100000)
          erros = erros + 1
    acumulador = acumulador + soma
    resumo_acumulado.append(acumulador)

portcentagem = acertos*100/janela
print("Acertos: ", acertos)
print("Erros: ", erros)
print("Pontos Ganhos: ", acumulador)
print("%", portcentagem)
print("Meses em teste estudados: ", int(len(predicted_price)/janela))
# resumo = pd.DataFrame(data=lucro_prejuizo , columns=['Resumo'])
resumo_acumulado = pd.DataFrame(data=resumo_acumulado , columns=['Resumo Acumulado '])
#return acumulador



plt.plot(resumo_acumulado, color='blue', label='Preço Atual das Ações')
plt.title('Resumo Acumulado')
plt.xlabel('Tempo')
plt.ylabel('Preço das Ações')
plt.legend()
plt.show()
plt.title('Preço final')
plt.xlabel('Tempo')
plt.ylabel('Preço das Ações')
plt.plot(y_test, color='red', label='Preço Atual das Ações')
plt.show()

In [ ]:
maior_ponto   =  0
rotacao       =  0
maior_acerto      =  0

#for epochs in range(30,50):
for i in range(1):
   model = treinamento_rede_neural(timestamp, features, x_train, y_train, epochs)
   pontos,acerto,data_test = teste_rede(model ,df_test, df_test_orig, sc, janela, shift )
   if acerto > maior_acerto:
     if pontos > maior_ponto:
       #model.save('drive/My Drive/TCC/modelo/LSTM')
       save_tocsv(data_test)
       maior_ponto = pontos
       maior_acerto = acerto

print(maior_ponto, maior_acerto)